In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import torch

In [4]:
df = pd.read_csv('Data\goodreads_train.csv')

In [5]:
tokenizer = Tokenizer(num_words=None, oov_token='<UNK>', char_level=False)
tokenizer.fit_on_texts(df['review_text'])

df['review_text'] = tokenizer.texts_to_sequences(df['review_text'])

In [12]:
df['review_text'][0:2].values

array([list([13, 10, 6, 701, 14, 8, 272, 419, 16, 35, 2, 67, 497, 108, 11, 2, 493, 497, 8, 272, 5, 71, 163, 108, 2, 201, 497, 2468, 30, 290, 13, 10, 42, 4, 44, 35, 77, 829, 443, 8, 3664, 156, 416, 35, 128, 102, 80, 147, 8, 10, 6, 1777, 5500, 3345, 3, 6095, 45, 217, 528, 3289, 96, 120, 8, 163, 11, 32, 6, 185, 82, 45, 127, 102, 159, 31, 16, 2678, 2, 59453, 7, 3289, 7493, 489, 52, 43, 326, 15781, 111, 7, 112, 109635, 700, 8, 10, 6, 14, 35, 829, 3, 2270, 2, 829, 1087, 11, 2, 14, 10, 2516, 217, 6, 14, 5260, 11, 5621, 3, 184, 2455, 19, 283, 19, 4, 105, 322, 209, 124, 113, 51, 8, 151, 5, 18683, 60914, 85, 1434, 6599, 4, 81, 28, 12, 32, 408, 473, 48, 163, 5, 81, 35, 113, 15, 42, 64, 338, 57, 267, 60915, 16616, 877, 6, 1245, 57, 88, 169, 266, 12, 49, 46, 3, 2, 249, 21735, 3, 8791, 2, 9633, 25, 267, 349, 12, 13779, 9, 170, 7, 2, 14, 12, 6, 129, 332, 4, 64, 29, 5, 81, 414, 1479, 5, 2717, 1763, 5010, 9, 10, 7770, 64, 26, 40, 29, 15823, 772, 128, 88, 64, 1894, 202, 4, 91, 29, 52, 2, 14, 12978, 2, 4

In [11]:
torch_tensor = torch.tensor(df['review_text'][0:2].values)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [10]:
torch.nn.functional.pad(df['review_text'][0:2])

TypeError: pad(): argument 'input' (position 1) must be Tensor, not Series

In [ ]:
class DataPreprocessing():

    def __init__(self,df):
        self.df = df
        pass

    def tokenize_reviews(self, num_words):
        tokenizer = Tokenizer(num_words=None, oov_token='<UNK>', char_level=False)
        self.df['review_text'] = tokenizer.texts_to_sequences(self.df['review_text'])
    
    def pad_reviews(self)


In [6]:
model_input_dim = tokenizer.num_words

In [ ]:
class RatingModel(torch.nnn.Module):

    def __init__(self, model_input_dim):

        super(RatingModel,self).__init__()
        self.layer1 = torch.nn.Embeding(model_input_dim, 200)

    
    def forward(self,x):


In [103]:
df['review_text'].to_numpy()

(900000,)